# Estudo de Caso: Jogos de Magnus Carlsen

Uma das vantagens do xadrez online é que conseguimos facilmente rastrear os resultados de jogos passados. O arquivo `carlsen_games.csv` contém as estatísticas de todos os jogos do jogador Magnus Carlsen nos últimos anos, e vamos trabalhar com eles.

DISCLAIMER: Neste estudo de caso, desenvolva seu próprio código e seu próprio raciocínio. Se precisar, faça em grupos. Há uma solução proposta no fim dele, mas ela só deve ser usada para conferir seus resultados - ou para comparar seu código. Em hipótese alguma você deve ver a solução antes de fazer sua própria solução. O motivo disso é que uma parte do motivo de fazer o estudo de caso é entender como buscar as próprias respostas. Então, use a solução apenas depois de ser capaz de julgar se ela está correta ou não. Para garantir que isso será feito, propositalmente coloquei (ou não) alguns erros na solução, então sempre veja a solução mostrada como um olhar crítico. A solução mostrada também não é a única. Proponha otimizações!!!


## Enunciado

Aqui temos dois arquivos: `carlsen_games.csv` e `user_names.csv`. Gostaríamos de responder às seguintes perguntas:

1. Quais são os usernames usados pelo Magnus Carlsen?
1. Tipicamente, dizemos que jogar com as brancas leva a maior chance de vitória que jogar com as pretas. Isso se aplica aos jogos do Magnus Carlsen?
1. Qual é a probabilidade de vitória anual do Magnus Carlsen, isto é, a probabilidade de vitória a cada ano quando os jogos são agrupados por ano?
1. De acordo com o resultado acima, seria correto modelar o número de vitórias mensal do Magnus Carlsen usando uma única distribuição Binomial? Explique por que a Binomial faz sentido para esse modelo, mas não seria correto usar os mesmos parâmetros para todos os meses.
1. O [Elo Rating](https://en.wikipedia.org/wiki/Elo_rating_system) é um sistema de pontuação que é usado para rankear jogadores de xadrez: quanto melhor o jogador, melhor seu Elo. Usando um teste de hipóteses adequado, discuta: seria razoável dizer que os jogadores que ganham do Magnus Carlsen tipicamente têm um Elo maior que os que perdem?

In [61]:
# Resolva aqui

In [ ]:
#




















# -----------------------

### SOLUÇÃO ESTÁ ABAIXO!!!


# ------------------------



In [57]:
import pandas as pd

df_usernames = pd.read_csv('user_names.csv', encoding='latin1')
df = pd.read_csv('carlsen_games.csv')
df = df[ df['time_control'] == '60+0' ]
magnus = list(df_usernames [df_usernames['real_name']=='Magnus Carlsen']['user_name'])
print(magnus)

['damnsaltythatsport', 'DannytheDonkey', 'DrChampionstein', 'DrDrunkenstein', 'DrGrekenstein', 'DrNykterstein', 'MagnusCarlsen', 'manwithavan', 'neslraCsungaM77', 'STL_Carlsen']


In [33]:
# Prob vitorias
carlsen_white = df[ df['white'].isin(magnus)]
carlsen_white_vitoria = df[ (df['white'].isin(magnus)) & (df['winner'].isin(magnus)) ]
p_vitoria_dado_white = len(carlsen_white_vitoria) / len(carlsen_white)

carlsen_black = df[ df['black'].isin(magnus) ]
carlsen_black_vitoria = df[ (df['black'].isin(magnus)) & (df['winner'].isin(magnus)) ]
p_vitoria_dado_black = len(carlsen_black_vitoria) / len(carlsen_black)

print(p_vitoria_dado_white, p_vitoria_dado_black)

0.7201754385964912 0.6521355453582774


In [50]:
def get_ano(s):
    return int(s.split('.')[0])
ano = df['date_played'].apply(get_ano)
df['ano'] = ano

p_vitorias_ano = dict()

for yr in df['ano'].unique():
    vitorias = df[ (df['ano']==yr) & (df['winner'].isin(magnus)) ]
    jogos = df[ df['ano']==yr ]
    p_vitoria_dado_ano = len(vitorias)/len(jogos)
    p_vitorias_ano[yr] = p_vitoria_dado_ano

p_vitorias_ano

{2018: 0.749691738594328,
 2017: 0.0,
 2021: 0.6829896907216495,
 2020: 0.6708265509133658,
 2019: 0.7268760907504364}

In [51]:
elo_perdedores = df[ df['winner'].isin(magnus)]['loser_elo']
elo_vencedores = df[ df['loser'].isin(magnus)]['winner_elo']

In [52]:
elo_perdedores.describe()

count    7800.000000
mean     2884.297692
std       267.881018
min       968.000000
25%      2815.000000
50%      2948.000000
75%      3053.000000
max      3269.000000
Name: loser_elo, dtype: float64

In [53]:
elo_vencedores.describe()

count    2691.000000
mean     2971.297287
std       168.911314
min      1714.000000
25%      2875.000000
50%      3007.000000
75%      3083.000000
max      3265.000000
Name: winner_elo, dtype: float64

In [60]:
import scipy.stats as st
st.ttest_ind(elo_perdedores, elo_vencedores, alternative='less')

TtestResult(statistic=-15.798383345696642, pvalue=6.964659304718891e-56, df=10489.0)